In [1]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

In [2]:
!pip install elasticsearch -q

     |████████████████████████████████| 225kB 2.6MB/s 


In [3]:
from subprocess import Popen, PIPE, STDOUT
from elasticsearch import Elasticsearch
from elasticsearch import helpers

import os
import time
import pprint

In [4]:
es_server = Popen(
  ['elasticsearch-7.0.0/bin/elasticsearch'], 
  stdout = PIPE, stderr = STDOUT,
  preexec_fn = lambda: os.setuid(1))

In [6]:
!curl -X GET "localhost:9200/"

{
  "name" : "4ba33636838f",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "XgAxmLD8SjW_Bf76o1Wk2w",
  "version" : {
    "number" : "7.0.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "b7e28a7",
    "build_date" : "2019-04-05T22:55:32.697037Z",
    "build_snapshot" : false,
    "lucene_version" : "8.0.0",
    "minimum_wire_compatibility_version" : "6.7.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [7]:
def gen_data():
  with open('/content/gdrive/My Drive/finch/es/free_chat/data/basic.txt') as f:
    for line in f:
      line = line.rstrip()
      q, a = line.split('<SEP>')
      yield {
        '_index': 'chatbot',
        'question': q,
        'answer': a,}

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
es = Elasticsearch()
print(es.ping())

es.indices.create(index='chatbot')

mapping = {
  'properties': {
    'question': {
      'type': 'text',
    }
  }
}
es.indices.put_mapping(body=mapping, index='chatbot')

helpers.bulk(es, gen_data())

True


(366, [])

In [ ]:
while True:
  text_inp = input('Input:')
  t0 = time.time()
  dsl = {
    'query': {
      'match': {
        'question': text_inp,
      }
    }
  }
  hits = es.search(index='chatbot', body=dsl)['hits']['hits']
  print('Output:', hits[0]['_source']['answer'])
  print('%.2f sec' % (time.time() - t0))
  print()

Input:爱你
Output: 爱你
0.10 sec

Input:我喜欢你
Output: 书是人类进步的阶梯
0.03 sec

Input:我爱你
Output: 我也爱你
0.01 sec

Input:天气
Output: 天气还行 要查具体的吗
0.01 sec

Input:几点了
Output: 要查一下现在的具体时间吗
0.01 sec

Input:几岁了
Output: 你猜呢
0.01 sec

Input:笑话
Output: 白天文明 但不精神 晚上精神 但不文明
0.01 sec

Input:天气
Output: 天气还行 要查具体的吗
0.01 sec

Input:天气好
Output: 适合出门走动走动
0.01 sec

Input:天气糟糕
Output: 最好不要出门
0.01 sec

Input:冷
Output: 多穿衣服哈
0.01 sec

Input:热
Output: 对啊, 热死人了
0.01 sec

Input:热死了
Output: 我也想死你了
0.01 sec

Input:激动死了
Output: 发生什么好事了 说来听听
0.01 sec

Input:我伤心了
Output: 怎么了 说来听听
0.01 sec

Input:厉害了
Output: 这么夸我我会骄傲的
0.01 sec

Input:我想回家
Output: 那就回呗
0.01 sec

